

<font size="4"> Iscte Instituto Universitário de Lisboa </font>
  
<font size="4"> Desenho e Análise de Algoritmos </font>
     
<font size="3"> Abril 2023 </font>

   
<font size="5"> <div class="alert alert-block alert-info"> **Experimentação - Aulas TP 15 e 16** </div></font> 
 


# Aulas Semana 8: Grafos e Travessias em Grafos


### Imports

In [1]:
import random as rnd


### Classes

In [2]:
# Class Vertice
class Vertex:
    '''Estrutura de Nó para um grafo: um elemento que é o identificador deste nó'''

    def __init__(self, x):
        '''O vértice será inserido no Grafo usando o método insert_vertex(x) que cria um Vertex'''
        self._elemento = x

    def __hash__(self):
        ''' este vértice (o seu identificador) é usado como chave'''
        return hash(id(self))  # devolve um inteiro que identifica este vértice como uma chave num dicionário

    def __str__(self):
        return'{0}'.format(self._elemento)

    def __eq__(self, x):
        return x == self._elemento

    def vertice(self):
        '''Devolve o nome deste vértice'''
        return self._elemento


# #### Class Edge
class Edge:
    '''Estrutura de Aresta para um Grafo: (origem, destino) e peso '''

    def __init__(self, u, v, p):
        self._ant = u
        self._suc = v
        self._weight = p

    def __hash__(self):
        # para associar a aresta a uma chave para um dicionário
        return hash( (self._ant, self._suc) )

    def __str__(self):
        '''Mostra a Aresta para um Grafo: (origem, destino) - peso '''
        return'({0},{1})-{2} '.format(self._ant, self._suc, self._weight)

    def __eq__(self, other):
        # define igualdade de duas arestas
        return self._ant == other._ant and self._suc == other._suc

    def endpoints(self):
        '''devolve (u,v) para indicar os vértices antecessor e sucessor.'''
        return (self._ant, self._suc)

    def opposite(self, v):
        '''Indica o vértice oposto ao v neste arco; v tem de ser um dos vértices.'''
        return self._suc if v is self._ant else self._ant

    def cost(self):
        '''Devolve o peso associado a este arco.'''
        return self._weight

    def show_edge(self):
        print('(',self._ant, ', ', self._suc, ') com peso', self._weight)       

In [3]:
#  ### Classe Grafo
class Graph(Vertex, Edge):
    '''Representação de um grafo usando mapeamentos de adjacências (associações) - dictionaries'''

    def __init__(self, directed=False):
        '''Cria um grafo vazio (dicionário de _vertices); é orientado se o parâmetro directed tiver o valor True'''
        self._directed = directed
        self._n = 0                 # quantidade de nós no Grafo
        self._m = 0                 # quantidade de arcos no Grafo
        self._vertices = {}         # dicionário com chave vértice e valor dicionário de adjacências

    def insert_vertex(self, x):
        '''Insere e devolve um novo vértice com o elemento x'''
        v = Vertex(x)
        self._vertices[v] = {}      # inicializa o dicionário de adjacências deste vértice a vazio
        self._n +=1                 # mais um vértice no grafo
        return v

    def insert_edge(self, u, v, x):
        '''Cria e insere uma nova aresta entre u e v com peso x'''
        e = Edge(u, v, x)
        self._vertices[u][v] = e  # vai colocar nas adjacências de u
        self._vertices[v][u] = e  # e nas adjacências de v (para facilitar a procura de todos os arcos incidentes em ou originários de)
        self._m +=1

    def is_directed(self):
        '''com base na criação original da instância, devolve True se o Grafo é dirigido; False senão '''
        return self._directed  # True se os dois contentores são distintos

    def order(self):
        '''Ordem de um grafo: a quantidade de vértices no Grafo'''
        return self._n

    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo'''
        return self._vertices.keys()

    def size(self):
        '''Dimensão de um grafo: a quantidade total de arestas do Grafo'''
        return self._m

    def set_of_edges(self):
        '''Devolve o conjunto (set) de todas as arestas do Grafo'''
        result = set()      # avoid double-reporting edges in undirected graph
        for secondary_map in self._vertices.values():
            result.update(secondary_map.values())  # add edges to resulting set
        return result


    def degree(self, v, outgoing=True):
        '''Quantidade de arestas originárias ou incidentes no vértice v 
           Se for um grafo dirigido, conta as arestas outgoing ou incoming, 
           de acordo com o valor de outgoing (True or False)
        '''
        adj = self._vertices
        if not self._directed:
            count = len(adj[v])
        else:
            count = 0
            for edge in adj[v].values():
                x, y = edge.endpoints()
                if (outgoing and x == v) or (not outgoing and y == v):
                    count += 1                
        return count


    def get_edge(self, u, v):
        '''Método interno: Devolve a aresta que liga u a v ou None se não forem adjacentes'''  
        edge = self._vertices[u].get(v)     # returns None se não existir aresta entre u e v
        if edge and self._directed: # se é dirigido
            x = edge.endpoints()        # vai confirmar se é o arco u --> v
            if x[1] != v:
                edge = None
        return edge
    
    def remove_edge(self, u, v):
        '''Remove a aresta entre u e v '''
        if  u in self._vertices.keys() and v in self._vertices[u].keys():
            del self._vertices[u][v]
            del self._vertices[v][u]
            self._m -=1

    def remove_vertex(self, v):
        '''remove o vértice v'''
        # remover todas as arestas de [v]
        # remover todas as arestas com v noutros vertices
        # remover o vértice v
        if v in self._vertices.keys():
            lst = [i for i in self.incident_edges(v)]
            for i in lst:
                x, y = i.endpoints()
                self.remove_edge(x,y)
            del self._vertices[v]
            self._n -=1
        #return v


    #outros métodos auxiliares#
    def incident_edges(self, v, incoming=True):
        '''Gerador: indica todas as arestas incoming de v
           Se for um grafo dirigido e incoming for False, devolve as arestas outgoing
        '''
        for edge in self._vertices[v].values(): # para todas as arestas relativas a v:
            if not self._directed:
                    yield edge
            else:  # senão deve ir procurar em todas as arestas para saber quais entram ou saiem
                x, y = edge.endpoints()
                if (incoming and y == v) or (not incoming and x == v):
                    yield edge

    def printG(self):
        '''Mostra o grafo por linhas'''
        if self._n == 0:
            print('O grafo está vazio!')
        else:
            print('Grafo orientado:', self._directed)
            for v in self.vertices():
                print('\nvertex ', v, ' grau_in: ', self.degree(v,False), end=' ')# mostra o grau (de entrada, se orientado)
                for i in self.incident_edges(v):
                    print(' ', i, end=' ')
                if self._directed:          # se orientado, mostrar o grau de saída
                    print('\n \t   grau_out: ', self.degree(v, True), end=' ')
                    for i in self.incident_edges(v, False):    # e mostra os arcos de saída
                        print(' ', i, end=' ')
            



#### Alguns testes à classe Graph

In [4]:
g = Graph() # Criar um Grafo não orientado
lst = [i for i in range(0, 10)]
vert = []                  # lista auxiliar para guardar os vértices inseridos para construção das arestas
for i in lst:
    vert.append(g.insert_vertex(i))  # inserção dos 10 vertices V = {0, 1, ..., 9} no grafo e na lista de vertices

rnd.seed(10)                # para futura replicação deste grafo
for i in range(1, 21):                      # criação de 20 arestas a partir dos vértices inseridos
    u, v = rnd.sample(lst, k=2)             # gerar aleatoriamente uma aresta entre 2 vértices deste grafo
    x = rnd.randint(1, 10)                  # com peso inteiro aleatório entre 0 e 10
    g.insert_edge(vert[u], vert[v], x)      # inserção desta aresta no grafo


In [5]:
g.printG()


Grafo orientado: False

vertex  0  grau_in:  4   (9,0)-4    (7,0)-4    (2,0)-9    (3,0)-1  
vertex  1  grau_in:  0 
vertex  2  grau_in:  5   (2,0)-9    (6,2)-10    (2,4)-6    (2,7)-4    (2,3)-7  
vertex  3  grau_in:  3   (3,5)-1    (2,3)-7    (3,0)-1  
vertex  4  grau_in:  2   (4,9)-8    (2,4)-6  
vertex  5  grau_in:  5   (7,5)-2    (3,5)-1    (5,6)-7    (5,8)-8    (9,5)-9  
vertex  6  grau_in:  3   (6,2)-10    (5,6)-7    (6,7)-2  
vertex  7  grau_in:  5   (7,0)-4    (7,9)-5    (7,5)-2    (2,7)-4    (6,7)-2  
vertex  8  grau_in:  1   (5,8)-8  
vertex  9  grau_in:  4   (9,0)-4    (7,9)-5    (4,9)-8    (9,5)-9  

In [6]:
# teste à remoção de vértices e arestas
g.remove_vertex(vert[2])                  
g.remove_edge(vert[6],vert[7])
g.printG()


Grafo orientado: False

vertex  0  grau_in:  3   (9,0)-4    (7,0)-4    (3,0)-1  
vertex  1  grau_in:  0 
vertex  3  grau_in:  2   (3,5)-1    (3,0)-1  
vertex  4  grau_in:  1   (4,9)-8  
vertex  5  grau_in:  5   (7,5)-2    (3,5)-1    (5,6)-7    (5,8)-8    (9,5)-9  
vertex  6  grau_in:  1   (5,6)-7  
vertex  7  grau_in:  3   (7,0)-4    (7,9)-5    (7,5)-2  
vertex  8  grau_in:  1   (5,8)-8  
vertex  9  grau_in:  4   (9,0)-4    (7,9)-5    (4,9)-8    (9,5)-9  

In [7]:
# teste à remoção total (seria preferível fazer um método clear_graph...)
for i in lst:
    g.remove_vertex(vert[i])
g.printG()


O grafo está vazio!


In [8]:
# Agora um teste a um grafo orientado:
g1 = Graph(True)
vert = []    
for i in lst:
    vert.append(g1.insert_vertex(i))  # inserção de 10 vertices V = {0, 1, ..., 9} no grafo e na lista de vertices
    
rnd.seed(170)                # para futura replicação deste grafo
for i in range(1, 41):                      # criação de 40 arestas a partir dos vértices inseridos
    u, v = rnd.sample(lst, k=2)             # gerar aleatoriamente uma aresta entre 2 vértices deste grafo
    x = rnd.randint(1, 10)                  # com peso inteiro aleatório entre 0 e 10
    g1.insert_edge(vert[u], vert[v], x)      # inserção desta aresta no grafo
    
g1.printG()

Grafo orientado: True

vertex  0  grau_in:  3   (3,0)-6    (5,0)-5    (8,0)-6  
 	   grau_out:  3   (0,6)-9    (0,9)-8    (0,7)-2  
vertex  1  grau_in:  3   (4,1)-4    (5,1)-9    (9,1)-9  
 	   grau_out:  2   (1,3)-2    (1,8)-10  
vertex  2  grau_in:  3   (7,2)-3    (9,2)-9    (3,2)-5  
 	   grau_out:  1   (2,8)-6  
vertex  3  grau_in:  4   (1,3)-2    (6,3)-3    (5,3)-5    (7,3)-10  
 	   grau_out:  2   (3,0)-6    (3,2)-5  
vertex  4  grau_in:  0 
 	   grau_out:  4   (4,1)-4    (4,7)-2    (4,8)-3    (4,9)-5  
vertex  5  grau_in:  2   (7,5)-3    (6,5)-2  
 	   grau_out:  5   (5,9)-6    (5,8)-1    (5,0)-5    (5,1)-9    (5,3)-5  
vertex  6  grau_in:  2   (0,6)-9    (8,6)-1  
 	   grau_out:  2   (6,5)-2    (6,3)-3  
vertex  7  grau_in:  2   (4,7)-2    (0,7)-2  
 	   grau_out:  3   (7,5)-3    (7,2)-3    (7,3)-10  
vertex  8  grau_in:  4   (5,8)-1    (1,8)-10    (2,8)-6    (4,8)-3  
 	   grau_out:  3   (8,0)-6    (8,9)-2    (8,6)-1  
vertex  9  grau_in:  4   (5,9)-6    (0,9)-8    (8,9)-2    

In [9]:
print(g1.order())


10


In [10]:
g1.size()

40

In [11]:
# teste à remoção de vértices e arestas                 
g1.remove_edge(vert[8],vert[6])
g1.printG()



Grafo orientado: True

vertex  0  grau_in:  3   (3,0)-6    (5,0)-5    (8,0)-6  
 	   grau_out:  3   (0,6)-9    (0,9)-8    (0,7)-2  
vertex  1  grau_in:  3   (4,1)-4    (5,1)-9    (9,1)-9  
 	   grau_out:  2   (1,3)-2    (1,8)-10  
vertex  2  grau_in:  3   (7,2)-3    (9,2)-9    (3,2)-5  
 	   grau_out:  1   (2,8)-6  
vertex  3  grau_in:  4   (1,3)-2    (6,3)-3    (5,3)-5    (7,3)-10  
 	   grau_out:  2   (3,0)-6    (3,2)-5  
vertex  4  grau_in:  0 
 	   grau_out:  4   (4,1)-4    (4,7)-2    (4,8)-3    (4,9)-5  
vertex  5  grau_in:  2   (7,5)-3    (6,5)-2  
 	   grau_out:  5   (5,9)-6    (5,8)-1    (5,0)-5    (5,1)-9    (5,3)-5  
vertex  6  grau_in:  1   (0,6)-9  
 	   grau_out:  2   (6,5)-2    (6,3)-3  
vertex  7  grau_in:  2   (4,7)-2    (0,7)-2  
 	   grau_out:  3   (7,5)-3    (7,2)-3    (7,3)-10  
vertex  8  grau_in:  4   (5,8)-1    (1,8)-10    (2,8)-6    (4,8)-3  
 	   grau_out:  2   (8,0)-6    (8,9)-2  
vertex  9  grau_in:  4   (5,9)-6    (0,9)-8    (8,9)-2    (4,9)-5  
 	   grau_ou

In [12]:
print('Ordem:', g1.order(),'Tamanho:',g1.size())

Ordem: 10 Tamanho: 39


In [13]:
print(g1.get_edge(vert[8],vert[6]))

None


In [14]:
print(g1.get_edge(vert[9],vert[1]))

(9,1)-9 
